In [ ]:
import os

import pandas as pd
%matplotlib inline

import ukpopulation.utils as ukpoputils
import ukpopulation.snhpdata as SNHPData

# Read in the raw data

In [ ]:
data_path = os.path.join('.', 'data_as_provided')
output_path = os.path.join('.', 'data_processed')

keys = ['baseline', '0-unplanned', '1-new-cities', '2-expansion', '3-new-cities23', '4-expansion23', '5-new-cities23-nb', '6-new-cities30-nb']

filenames = {
    'baseline': os.path.join(data_path, 'Scenario Baseline - Dwelling+Employment projections.xlsx'), 
    '0-unplanned': os.path.join(data_path, 'Scenario Unplanned Development - Dwelling+Employment projections.xlsx'), 
    '1-new-cities': os.path.join(data_path, 'Scenario New Settlements - Dwelling+Employment projections.xlsx'), 
    '2-expansion': os.path.join(data_path, 'Scenario Expansion - Dwelling+Employment projections.xlsx'), 
    '3-new-cities23': os.path.join(data_path, 'Scenario New Settlements 23000.xlsx'), 
    '4-expansion23': os.path.join(data_path, 'Scenario Expansion 23000.xlsx'),
    '5-new-cities23-nb': os.path.join(data_path, 'Scenario New Settlements 23000_no_base_change.xlsx'),
    '6-new-cities30-nb': os.path.join(data_path, 'Scenario New Settlements 30000_no_base_change.xlsx'),
}

sheets = {
    'baseline': 'Projection dwelling baseline',
    '0-unplanned':  'Projection UnplannedDev',
    '1-new-cities': 'Projection New Settlem data',
    '2-expansion': 'Projection Expansion data',
    '3-new-cities23': 'Projection New Settlem data',
    '4-expansion23': 'Projection Expansion data',
    '5-new-cities23-nb': 'Projection New Settlem data',
    '6-new-cities30-nb': 'Projection New Settlem data',
}

In [ ]:
def get_households(year, geogs, df): 
    """Use UKPopulation to obtain historical and extrapolated household data for all other LADs
    
    Arguments
    ---------
    year : int
    geogs : list
    df : pandas.DataFrame
        The raw data
    
    Returns
    -------
    pandas.DataFrame
    """

    geogs = ukpoputils.split_by_country(geogs)

    allsnhp = pd.DataFrame()

    for country in geogs:
        if not geogs[country]: continue
        max_year = df.max_year(country)
        min_year = df.min_year(country)

        if year <= max_year:
            if year <= min_year:
                snhp = df.aggregate(geogs[country], min_year+1).merge(
                            df.aggregate(geogs[country], min_year), 
                            left_on="GEOGRAPHY_CODE", right_on="GEOGRAPHY_CODE")
                snhp["HOUSEHOLDS"] = snhp.OBS_VALUE_y + (snhp.OBS_VALUE_y - snhp.OBS_VALUE_x) * (min_year - year)
                snhp["PROJECTED_YEAR_NAME"] = year
                snhp.drop(["PROJECTED_YEAR_NAME_x", "OBS_VALUE_x", "PROJECTED_YEAR_NAME_y", "OBS_VALUE_y"], axis=1, inplace=True)
            else:
                snhp = df.aggregate(geogs[country], year).rename({"OBS_VALUE": "HOUSEHOLDS"}, axis=1)
        else:
            snhp = df.aggregate(
                geogs[country], max_year-1).merge(
                df.aggregate(geogs[country], max_year), 
                left_on="GEOGRAPHY_CODE", right_on="GEOGRAPHY_CODE")
            snhp["HOUSEHOLDS"] = snhp.OBS_VALUE_y + (snhp.OBS_VALUE_y - snhp.OBS_VALUE_x) * (year - max_year)
            snhp["PROJECTED_YEAR_NAME"] = year
            snhp.drop(["PROJECTED_YEAR_NAME_x", "OBS_VALUE_x", "PROJECTED_YEAR_NAME_y", "OBS_VALUE_y"], axis=1, inplace=True)

        allsnhp = allsnhp.append(snhp, ignore_index=True, sort=False)

    return allsnhp

In [ ]:
def process_df(df):
    df = df.dropna(
        ).reset_index(
        ).melt(
            id_vars='Area Name',
            var_name='timestep',
            value_name='dwellings')
    df = df.set_index(['Area Name', 'timestep'])
    return df

lad_nmcds = pd.read_csv(os.path.join(data_path, 'lad_nmcd_changes.csv'))
lad_nmcds = lad_nmcds[['lad16nm', 'lad16cd']]
all_lad_nms = set(lad_nmcds.lad16nm)

In [ ]:
def add_national_data(df, lad_nmcds, snhp):
    all_lads = set(lad_nmcds['lad16cd'])
    arc_lads = set(df.reset_index()['lad16cd'].unique())
    required_lads = all_lads - arc_lads

    years = list(df['timestep'].unique())
    dfs = [df]
    for year in years:
        national_data = get_households(year, required_lads, snhp)
        national_data = national_data.rename(columns={'GEOGRAPHY_CODE': 'lad16cd',
                                      'PROJECTED_YEAR_NAME': 'timestep',
                                      'HOUSEHOLDS': 'dwellings'})
        dfs.append(national_data)
    df = pd.concat(dfs, sort=False)
    df = df.rename(columns={'lad16cd': 'lad_uk_2016'})
    return df

In [ ]:
def add_lad_codes(df):

    baseline_lad_nms = set(df.reset_index()['Area Name'].unique())
    df_wlad = df.reset_index().merge(lad_nmcds, 
                                     left_on='Area Name', 
                                     right_on='lad16nm').drop(columns='Area Name')
    return df_wlad

In [ ]:
def write_out(df, filename):
    df.to_csv(os.path.join(output_path, filename), index=False)

In [ ]:
snhp = SNHPData.SNHPData()

In [ ]:
def read_excel_into_df(filename, sheet_name):
    
    years = [str(x) for x in range(2004, 2050)]
    types = {str(year): int for year in years}
    
    return pd.read_excel(
        filename, 
        sheet_name=sheet_name, 
        header=3, 
        index_col=0,
        names=['Area Name'].extend(years),
        nrows=26,  # magic number of rows
        dtype=types
    )

In [ ]:
data = {}
for scenario in keys:
    df = read_excel_into_df(filenames[scenario], sheets[scenario])
    df = process_df(df)
    df = add_lad_codes(df)
    data[scenario] = df

In [ ]:
arc_lads = set(data['baseline'].reset_index()['lad16nm'].unique())
assert len(arc_lads) == 26

In [ ]:
for scenario in keys:
    assert len(data[scenario][data[scenario].timestep.isin([2015,2050])]) == len(arc_lads)*2, scenario

In [ ]:
data_national = {}
for key, df in data.items():
    df = add_national_data(df, lad_nmcds, snhp)
    data_national[key] = df
    write_out(df, 'arc_dwellings__{}.csv'.format(key))

## Check the data

In [ ]:
def filter_on_arc(df, lad_nmcds, arc_lads):   
    just_arc_lads = df.lad_uk_2016.isin(lad_nmcds[lad_nmcds.lad16nm.isin(arc_lads)].lad16cd)
    
    df = df[just_arc_lads]
    df = df.set_index(['timestep', 'lad_uk_2016'])
    return df

In [ ]:
bas = filter_on_arc(data_national['baseline'], lad_nmcds, arc_lads)
bas['unplanned'] = filter_on_arc(data_national['0-unplanned'], lad_nmcds, arc_lads)['dwellings']
bas['settlements'] = filter_on_arc(data_national['1-new-cities'], lad_nmcds, arc_lads)['dwellings']
bas['expansion'] = filter_on_arc(data_national['2-expansion'], lad_nmcds, arc_lads)['dwellings']
bas['settlements23'] = filter_on_arc(data_national['3-new-cities23'], lad_nmcds, arc_lads)['dwellings']
bas['expansion23'] = filter_on_arc(data_national['4-expansion23'], lad_nmcds, arc_lads)['dwellings']
bas['settlements23nb'] = filter_on_arc(data_national['5-new-cities23-nb'], lad_nmcds, arc_lads)['dwellings']
bas['settlements30nb'] = filter_on_arc(data_national['6-new-cities30-nb'], lad_nmcds, arc_lads)['dwellings']
bas = bas.rename(columns={'dwellings': 'baseline'})

In [ ]:
df = bas[['expansion', 'expansion23', 'unplanned', 'baseline']]
df.groupby(by='timestep').sum().plot()

In [ ]:
df = bas[['settlements', 'settlements23']]
df.groupby(by='timestep').sum().plot()

In [ ]:
df = bas[['settlements30nb', 'settlements23nb']]
df.groupby(by='timestep').sum().plot()

In [ ]:
"Number of LADs in data: {}".format(len(data_national['baseline'].lad_uk_2016.unique()))

In [ ]:
baseline = data_national['baseline']
unplanned = data_national['0-unplanned']
settlements = data_national['1-new-cities']
expansion = data_national['2-expansion']
settlements23 = data_national['3-new-cities23']
expansion23 = data_national['4-expansion23']
settlements23nb = data_national['5-new-cities23-nb']
settlements30nb = data_national['6-new-cities30-nb']

baseline['scenario'] = 'baseline'
expansion['scenario'] = 'expansion'
settlements['scenario'] = 'settlements'
unplanned['scenario'] = 'unplanned'
settlements23['scenario'] = 'settlements23'
expansion23['scenario'] = 'expansion23'
settlements23nb['scenario'] = 'settlements23-nb'
settlements30nb['scenario'] = 'settlements-nb'
df = pd.concat([baseline, expansion, settlements, unplanned, expansion23, settlements23, settlements23nb, settlements30nb])
df.head()

In [ ]:
pivoted = df.groupby(
    ['timestep','scenario']
).sum().reset_index().pivot(
    index='timestep', columns='scenario', values='dwellings'
)
pivoted.plot()

In [ ]:
df.head()

In [ ]:
pivoted = pd.pivot_table(
    df, index=['timestep', 'lad_uk_2016'], columns='scenario', values='dwellings'
).reset_index().set_index('timestep')
pivoted.head()

In [ ]:
pivoted[pivoted.lad_uk_2016 =='E07000178'].plot()